In [9]:
import cv2
import numpy as np
import tensorflow as tf

In [10]:
def process_stream_image(image):
    """
    カメラ画像を前処理してモデル入力用に準備する関数
    """
    # リサイズ（モデルの入力サイズに合わせる）
    resized = cv2.resize(image, (128, 128))
    
    # 二値化処理（手の領域を抽出）
    _, binary = cv2.threshold(resized, 120, 255, cv2.THRESH_BINARY_INV)
    
    # ノイズ除去
    kernel = np.ones((3,3), np.uint8)
    cleaned = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
    
    # 正規化（0-1の範囲に）
    normalized = cleaned / 255.0
    
    return normalized

In [11]:
model = tf.keras.models.load_model('realtime_fingers_detection.keras')

# 必要に応じて重みを読み込む
WEIGHTS_FILE_NAME = 'realtime_fingers_weights.weights.h5'
model.load_weights(WEIGHTS_FILE_NAME)

# テストデータがない場合は評価をスキップ
# loss, acc = model.evaluate(X_test, Y_test)
# print(f"Model evaluated with Loss: {loss:.4f}, Accuracy: {acc:.4f}")

# モデルの入力形状を確認
print(f"モデルの入力形状: {model.input_shape}")

モデルの入力形状: (None, 128, 128, 1)


In [12]:
class FingerClassifier(object):
    def __init__(self, model_object):
        self.detect = model_object

    def get_classification(self, img):
        img = cv2.resize(img, (48, 48))
        img = img.reshape(1, img.shape[0], img.shape[1], 1)
        img = tf.constant(img, dtype=float)
        
        flat_img = img.numpy().flatten()
        unique, counts = np.unique(img, return_counts=True)
        if (len(counts) <= 1 or counts[1] < 1200):
            return -1;
        output = self.detect(img)
        return np.argmax(output)

In [13]:
obj = FingerClassifier(model)

In [14]:
camMarginX = 10
camMarginY = 10
scale = 10
LB = np.array([0, 90, 0])
UB = np.array([180, 220, 255])    
roi = (400, 120, 250, 250)
rval = True

cam = cv2.VideoCapture(0)

while rval:
    rval, img = cam.read()
    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    x, y, w, h = roi
    cropped = img[y:y+h, x:x+w]
    
    cv2.imshow('Original', cropped)
    cropped = process_stream_image(cropped)
    # show_image(cropped)
    cv2.imshow('Mask', cropped)
    fingers = obj.get_classification(cropped)
    print(fingers, end=' ')
    if(cv2.waitKey(25) & 0xFF == 27):
        break;

cam.release()
cv2.destroyAllWindows()

-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 2304, but received input with shape (1, 64)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 48, 48, 1), dtype=float32)
  • training=None
  • mask=None
  • kwargs=<class 'inspect._empty'>